In [17]:
import pandas as pd
import numpy as np
import re

In [15]:
df_raw = pd.read_csv("/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/_roads.tsv", sep='\t')

df_bridges_raw = pd.read_excel('/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/BMMS_overview.xlsx')

/var/folders/04/yj5x8lbx49sc1jbf3gyzwr3h0000gn/T/ipykernel_43399/1161747910.py:1: DtypeWarning: Columns (571,574,577,580,583,586,589,592,595,598,601,604,607,610,613,616,619,622,625,628,631,634,637,640,643,646,649,652,655,658,661,664,667,670,673,676,679,682,685,688,691,694,697,700,703,706,709,712,715,718,721,724,727,730,733,736,739,742,745,748,751,754,757,760,763,766,769,772,775,778,781,784,787,790,793,796,799,802,805,808,811,814,817,820,823,826,829,832,835,838,841,844,847,850,853,856,859,862,865,868,871,874,877,880,883,886,889,892,895,898,901,904,907,910,913,916,919,922,925,928,931,934,937,940,943,946,949,952,955,958,961,964,967,970,973,976,979,982,985,988,991,994,997,1000,1003,1006,1009,1012,1015,1018,1021,1024,1027,1030,1033,1036,1039,1042,1045,1048,1051,1054,1057,1060,1063,1066,1069,1072,1075,1078,1081,1084,1087,1090,1093,1096,1099,1102,1105,1108,1111,1114,1117,1120,1123,1126,1129,1132,1135,1138,1141,1144,1147,1150,1153,1156,1159,1162,1165,1168,1171,1174,1177,1180,1183,1186,1189,119

In [12]:
# 2. Inspect the columns
print("Columns found:", df_bridges_raw.columns.tolist())

# 3. Quick Clean (Standardize names)
# It's good practice to strip whitespace from column names immediately
df_bridges_raw.columns = df_bridges_raw.columns.str.strip()

# Check the first few rows to ensure the header was picked up correctly
print(df_bridges_raw.head())

Columns found: ['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition', 'structureNr', 'roadName', 'chainage', 'width', 'constructionYear', 'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon', 'EstimatedLoc']
  road      km              type  LRPName               name  length  \
0   N1   1.800       Box Culvert  LRP001a                  .   11.30   
1   N1   4.925       Box Culvert  LRP004b                  .    6.60   
2   N1   8.976  PC Girder Bridge  LRP008b  Kanch pur Bridge.  394.23   
3   N1  10.880       Box Culvert  LRP010b   NOYAPARA CULVERT    6.30   
4   N1  10.897       Box Culvert  LRP010c     ADUPUR CULVERT    6.30   

  condition  structureNr                                           roadName  \
0         A       117861  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...   
1         A       117862  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...   
2         A       119889  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...   
3         A       

In [23]:
import pandas as pd

# 1. LOAD THE FILE
# The file likely uses comma delimiters.
input_path = '/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/Roads_InfoAboutEachLRP.csv' # Adjust path if needed
print(f"Loading {input_path}...")

df = pd.read_csv(input_path, low_memory=False)

# 2. CLEAN UP DATA TYPES
# Ensure chainage is a number (sometimes it has typos)
df['chainage'] = pd.to_numeric(df['chainage'], errors='coerce')

# Ensure Lat/Lon are numbers
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

# 3. THE "FENCE" (Filter Bad Coordinates)
# Keep only points inside Bangladesh (Lat 20-27, Lon 88-93)
mask_lat = (df['lat'] > 20) & (df['lat'] < 27)
mask_lon = (df['lon'] > 88) & (df['lon'] < 93)
df_clean = df[mask_lat & mask_lon].copy()

# 4. THE "SORTER" (Fixes the Zig-Zags)
# Sort by Road Name, then by Chainage
df_clean = df_clean.sort_values(by=['road', 'chainage'])

# 5. REMOVE DUPLICATES (Fixes Loops/Knots)
df_clean = df_clean.drop_duplicates(subset=['road', 'chainage'])

# 6. SAVE IT BACK
# We overwrite the original file so your Java code sees the clean version.
# index=False removes row numbers.
# quoting=1 ensures values are wrapped in quotes "Value", just like the original file.
import csv
df_clean.to_csv(input_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

print(f"Success! Overwrote {input_path} with sorted, clean data.")
print(f"Original rows: {len(df)}")
print(f"Cleaned rows: {len(df_clean)}")
print(df_clean.head())

Loading /Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/Roads_InfoAboutEachLRP.csv...
Success! Overwrote /Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/Roads_InfoAboutEachLRP.csv with sorted, clean data.
Original rows: 52210
Cleaned rows: 51926
  road  chainage     lrp        lat        lon       type  \
0   N1     0.000    LRPS  23.706028  90.443333     Others   
1   N1     0.814   LRPSa  23.702917  90.450417    Culvert   
2   N1     0.822   LRPSb  23.702778  90.450472  CrossRoad   
3   N1     1.000  LRP001  23.702139  90.451972     KmPost   
4   N1     2.000  LRP002  23.697889  90.460583     KmPost   

                                                name  
0  Start of Road  after Jatrabari Flyover infront...  
1                                        Box Culvert  
2                            Intersection with Z1101  
3                                    Km post missing  
4                                    Km post missing  
